In [7]:
import pandas as pd
import requests

In [8]:
import geopandas as gpd
import zipfile
import io

In [2]:
# Variables in structured dict

vars_struct = {
    'Total Population' : "B01003_001E",
    'Population - Race': {
        "White Population" : "B02001_002E",
        "Black Population" : 'B02001_003E',
        "Hispanic or Latino Population": "B03003_003E",
        "Arab Population": 'B04004_006E',
        "American Indiana & Alaska Native Population": 'B02001_004E',
        "Asian Population" : 'B02001_005E',
        "Hawaiian & Pacific Islander Population": "B02001_006E"
    },
    'Economic':{
        'Income Total Population': 'B17001_001E',
        'Income Below Poverty Line': 'B17001_002E',
        'Median Income': 'B19013_001E',
        'Education Attainment 18 to 64 Total': 'B23006_001E',
        'Education Attainment Less than Highschool':'B23006_002E',
        'Education Attainment Highschool Graduate': 'B23006_009E',
        'Education Attainment Bachelors Degree or Higher': 'B23006_023E',
    },
    'Population - Sex': {
        'Male Population' : 'B01001_002E',
        "Female Population" : 'B01001_026E'
    },
    'Population - Age' : {
        'Male': {
            '0-5':'B01001_003E',
            '5-9':'B01001_004E',
            '10-14':'B01001_005E',
            '15-17':'B01001_006E',
            '18-19':'B01001_007E',
            '20':'B01001_008E',
            '21':'B01001_009E',
            '22-24':'B01001_010E',
            '25-29':'B01001_011E',
            '30-34':'B01001_012E',
            '35-39':'B01001_013E',
            '40-44':'B01001_014E',
            '45-49':'B01001_015E',
            '50-54':'B01001_016E',
            '55-59':'B01001_017E',
            '60-61':'B01001_018E',
            '62-64':'B01001_019E',
            '65-66':'B01001_020E',
            '67-69':'B01001_021E',
            '70-74':'B01001_022E',
            '75-79':'B01001_023E',
            '80-84':'B01001_024E',
            '85+': 'B01001_025E'

        },
        'Female': {
            '0-5':'B01001_027E',
            '5-9':'B01001_028E',
            '10-14':'B01001_029E',
            '15-17':'B01001_030E',
            '18-19':'B01001_031E',
            '20':'B01001_032E',
            '21':'B01001_033E',
            '22-24':'B01001_034E',
            '25-29':'B01001_035E',
            '30-34':'B01001_036E',
            '35-39':'B01001_037E',
            '40-44':'B01001_038E',
            '45-49':'B01001_039E',
            '50-54':'B01001_040E',
            '55-59':'B01001_041E',
            '60-61':'B01001_042E',
            '62-64':'B01001_043E',
            '65-66':'B01001_044E',
            '67-69':'B01001_045E',
            '70-74':'B01001_046E',
            '75-79':'B01001_047E',
            '80-84':'B01001_048E',
            '85+':'B01001_049E',
        }
    }
}

In [3]:
# Functions for retrieving & manipulating data


def get_data(year, dsource, dname, cols, state, county, tract, rename):

    year = year
    dsource = dsource
    dname = dname
    cols = cols
    state = state
    county = county
    tract = tract

    base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}/'

    data_url = f'{base_url}?get={cols}&for=tract:{tract}&in=state:{state}&in=county:{county}'

    response = requests.get(data_url)
    data = pd.DataFrame(columns=response.json()[0], data=response.json()[1:])
    
    data['GEOID'] = data['state'] + data['county'] + data['tract']
    data = data.drop(columns =['NAME','state','county','tract'])
    data = data.set_index('GEOID')
        
    data.columns = rename
    data = data.astype(int)
    data = data.sort_index()
    
    return data

def get_name(year, dsource, dname, cols, state, county, tract):

    year = year
    dsource = dsource
    dname = dname
    cols = cols
    state = state
    county = county
    tract = tract

    base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}/'

    data_url = f'{base_url}?get={cols}&for=tract:{tract}&in=state:{state}&in=county:{county}'

    response = requests.get(data_url)
    data = pd.DataFrame(columns=response.json()[0], data=response.json()[1:])
    
    data['GEOID'] = data['state'] + data['county'] + data['tract']
    data = data.set_index('GEOID')
    
    data[['Census Tract', 'Country','State']] = data['NAME'].str.split(",", n = 2, expand = True)
    data = data.drop(columns = ['state','county','tract','NAME'])

    data = data.sort_index()
    
    return data

def pop_condense(data, gender):
    df = data.copy()
    df[f'{gender} Pop Below 18'] = df.iloc[:,0:4].astype(int).sum(axis=1)
    df[f'{gender} Pop 18 to 24'] = df.iloc[:,4:8].astype(int).sum(axis=1)
    df[f'{gender} Pop 25 to 34'] = df.iloc[:,8:10].astype(int).sum(axis=1)
    df[f'{gender} Pop 35 to 44'] = df.iloc[:,10:12].astype(int).sum(axis=1)
    df[f'{gender} Pop 45 to 54'] = df.iloc[:,12:14].astype(int).sum(axis=1)
    df[f'{gender} Pop 55 to 64'] = df.iloc[:,14:17].astype(int).sum(axis=1)
    df[f'{gender} Pop 65+'] = df.iloc[:,17:23].astype(int).sum(axis=1)
    return df

def to_percent(data,total_data):
    df = data.copy()
    tots = total_data.copy()
    
    vals = df.values / tots.values
    
    return pd.DataFrame(vals, columns = df.columns, index = df.index)
    



In [26]:
# Single Data pull walkthrough

# Total Population

total_pop = vars_struct['Total Population']
    
year = '2019'
dsource = 'acs'
dname = 'acs5'
cols = 'NAME,' + total_pop
state = '04'
county = '*'
tract = '*'
rename = ['Total Population']



total_ = get_data(year, dsource, dname, cols, state, county, tract, rename)

In [27]:
cols = 'NAME' 
names = get_name(year, dsource, dname, cols, state, county, tract)

In [33]:

# Male Population & Percent
col_names = ['Male ' + i for (i,v) in vars_struct['Population - Age']['Male'].items()]
col_codes = [v for (i,v) in vars_struct['Population - Age']['Male'].items()]

cols = 'NAME,' + ','.join(col_codes)
male_counts = get_data(year, dsource, dname, cols, state, county, tract, col_names)

male_counts = pop_condense(male_counts, 'Male')
male_perc = to_percent(male_counts, total_)

male_perc.columns = [i + ' Percent' for i in list(male_perc)]

# Female Population & Percent
col_names = ['Female ' + i for (i,v) in vars_struct['Population - Age']['Female'].items()]
col_codes = [v for (i,v) in vars_struct['Population - Age']['Female'].items()]

cols = 'NAME,' + ','.join(col_codes)
female_counts = get_data(year, dsource, dname, cols, state, county, tract, col_names)

female_counts = pop_condense(female_counts, 'Female')
female_perc = to_percent(female_counts,total_)

female_perc.columns = [i + ' Percent' for i in list(female_perc)]

<ipython-input-3-26d4bc556acf>:46: RuntimeWarning: invalid value encountered in true_divide
  vals = df.values / tots.values


In [46]:

total_population_by_age = pd.DataFrame(female_counts.values + male_counts.values, 
                                       index = total_.index,
                                       columns = male_counts.columns
                                      )

total_population_by_age.columns = [i.replace('Male ',"Total ") for i in list(total_population_by_age)]

In [35]:

# Econ Cleaning

col_names = [i for (i,v) in vars_struct['Economic'].items()]
col_codes = [v for (i,v) in vars_struct['Economic'].items()]

cols = 'NAME,' + ','.join(col_codes)
econ_ = get_data(year, dsource, dname, cols, state, county, tract, col_names)

econ_['Income Below Poverty Line %'] = econ_['Income Below Poverty Line'] / econ_['Income Total Population']
econ_['Education: % Less than Highschool'] =  econ_['Education Attainment Less than Highschool'] / econ_['Education Attainment 18 to 64 Total']
econ_['Education: % Highschool Graduate'] =  econ_['Education Attainment Highschool Graduate'] / econ_['Education Attainment 18 to 64 Total']
econ_['Education: % Bachelors or Higher'] =  econ_['Education Attainment Bachelors Degree or Higher'] / econ_['Education Attainment 18 to 64 Total']

In [36]:
col_names = [i for (i,v) in vars_struct['Population - Race'].items()]
col_codes = [v for (i,v) in vars_struct['Population - Race'].items()]

cols = 'NAME,' + ','.join(col_codes)
pop_race = get_data(year, dsource, dname, cols, state, county, tract, col_names)
pop_race_percent = to_percent(pop_race,total_)
pop_race_percent.columns = [i + ' Percent' for i in list(pop_race_percent)]

<ipython-input-3-26d4bc556acf>:46: RuntimeWarning: invalid value encountered in true_divide
  vals = df.values / tots.values


In [37]:
# Population by sex
col_names = [i for (i,v) in vars_struct['Population - Sex'].items()]
col_codes = [v for (i,v) in vars_struct['Population - Sex'].items()]

cols = 'NAME,' + ','.join(col_codes)
pop_sex = get_data(year, dsource, dname, cols, state, county, tract, col_names)
pop_sex_percent = to_percent(pop_sex,total_)
pop_sex_percent.columns = [i + ' Percent' for i in list(pop_sex_percent)]

<ipython-input-3-26d4bc556acf>:46: RuntimeWarning: invalid value encountered in true_divide
  vals = df.values / tots.values


In [50]:
out_data = total_.join([names,female_counts,female_perc,male_counts,male_perc,total_population_by_age,econ_,pop_race,pop_race_percent,pop_sex,pop_sex_percent])

out_data = out_data.fillna(0.0)

#out_data.to_csv(f'data_state_{state}.csv',compression='gzip')


In [6]:

def query_data(state_code, vars_struct):
    
    total_pop = vars_struct['Total Population']
    year = '2019'
    dsource = 'acs'
    dname = 'acs5'
    cols = 'NAME,' + total_pop
    state = state_code
    county = '*'
    tract = '*'
    rename = ['Total Population']
    
    total_ = get_data(year, dsource, dname, cols, state, county, tract, rename)
    
    cols = 'NAME' 
    names = get_name(year, dsource, dname, cols, state, county, tract)

    # Male Population & Percent
    col_names = ['Male ' + i for (i,v) in vars_struct['Population - Age']['Male'].items()]
    col_codes = [v for (i,v) in vars_struct['Population - Age']['Male'].items()]

    cols = 'NAME,' + ','.join(col_codes)
    male_counts = get_data(year, dsource, dname, cols, state, county, tract, col_names)

    male_counts = pop_condense(male_counts, 'Male')
    male_perc = to_percent(male_counts, total_)

    male_perc.columns = [i + ' Percent' for i in list(male_perc)]

    # Female Population & Percent
    col_names = ['Female ' + i for (i,v) in vars_struct['Population - Age']['Female'].items()]
    col_codes = [v for (i,v) in vars_struct['Population - Age']['Female'].items()]

    cols = 'NAME,' + ','.join(col_codes)
    female_counts = get_data(year, dsource, dname, cols, state, county, tract, col_names)

    female_counts = pop_condense(female_counts, 'Female')
    female_perc = to_percent(female_counts,total_)

    female_perc.columns = [i + ' Percent' for i in list(female_perc)]
    
    
    total_population_by_age = pd.DataFrame(female_counts.values + male_counts.values, 
                                           index = total_.index,
                                           columns = male_counts.columns
                                          )

    total_population_by_age.columns = [i.replace('Male ',"Total ") for i in list(total_population_by_age)]

    # Econ Cleaning

    col_names = [i for (i,v) in vars_struct['Economic'].items()]
    col_codes = [v for (i,v) in vars_struct['Economic'].items()]

    cols = 'NAME,' + ','.join(col_codes)
    econ_ = get_data(year, dsource, dname, cols, state, county, tract, col_names)

    econ_['Income Below Poverty Line %'] = econ_['Income Below Poverty Line'] / econ_['Income Total Population']
    econ_['Education: % Less than Highschool'] =  econ_['Education Attainment Less than Highschool'] / econ_['Education Attainment 18 to 64 Total']
    econ_['Education: % Highschool Graduate'] =  econ_['Education Attainment Highschool Graduate'] / econ_['Education Attainment 18 to 64 Total']
    econ_['Education: % Bachelors or Higher'] =  econ_['Education Attainment Bachelors Degree or Higher'] / econ_['Education Attainment 18 to 64 Total']

    
    col_names = [i for (i,v) in vars_struct['Population - Race'].items()]
    col_codes = [v for (i,v) in vars_struct['Population - Race'].items()]

    cols = 'NAME,' + ','.join(col_codes)
    pop_race = get_data(year, dsource, dname, cols, state, county, tract, col_names)
    pop_race_percent = to_percent(pop_race,total_)
    pop_race_percent.columns = [i + ' Percent' for i in list(pop_race_percent)]
    
    # Population by sex
    col_names = [i for (i,v) in vars_struct['Population - Sex'].items()]
    col_codes = [v for (i,v) in vars_struct['Population - Sex'].items()]

    cols = 'NAME,' + ','.join(col_codes)
    pop_sex = get_data(year, dsource, dname, cols, state, county, tract, col_names)
    pop_sex_percent = to_percent(pop_sex,total_)
    pop_sex_percent.columns = [i + ' Percent' for i in list(pop_sex_percent)]
    
    out_data = total_.join([names,female_counts,female_perc,male_counts,male_perc,total_population_by_age,econ_,pop_race,pop_race_percent,pop_sex,pop_sex_percent])
    out_data = out_data.fillna(0.0)
    
    out_data.to_csv(f'./data/storage/tabular_state_data/data_state_{state_code}.csv',compression='gzip')


    

In [8]:
query_data('04',vars_struct)

<ipython-input-3-2a5cafaa9bd3>:73: RuntimeWarning: invalid value encountered in true_divide
  vals = df.values / tots.values


In [15]:
pd.read_csv('data_state_12.csv',compression = 'gzip',index_col = 0)

,Total Population,Female 0-5,Female 5-9,Female 10-14,Female 15-17,Female 18-19,Female 20,Female 21,Female 22-24,Female 25-29,...,Black Population Percent,Hispanic or Latino Population Percent,Arab Population Percent,American Indian & Alaska Native Population Percent,Asian Population Percent,Hawaiian & Pacific Islander Population Percent,Male Population,Female Population,Male Population Percent,Female Population Percent
GEOID,,,,,,,,,,,,,,,,,,,,,
4001942600,1742,60,65,76,33,9,0,14,53,116,...,0.000000,0.000000,0.0,0.997130,0.000000,0.000000,855,887,0.490815,0.509185
4001942700,5345,157,136,201,181,79,36,57,120,103,...,0.001310,0.006361,0.0,0.965201,0.009355,0.000561,2772,2573,0.518616,0.481384
4001944000,6547,203,218,374,129,120,91,78,102,233,...,0.007790,0.011150,0.0,0.921338,0.012983,0.000000,3197,3350,0.488315,0.511685
4001944100,5987,264,257,246,138,258,77,63,204,214,...,0.002338,0.016202,0.0,0.954234,0.002338,0.000000,2774,3213,0.463337,0.536663
4001944201,4473,160,212,257,127,107,24,0,47,109,...,0.005366,0.003353,0.0,0.932484,0.000000,0.000000,2258,2215,0.504807,0.495193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027012100,1117,68,29,8,62,10,0,0,15,16,...,0.000895,0.538944,0.0,0.015219,0.000000,0.001791,644,473,0.576544,0.423456
4027980003,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0,0.000000,0.000000
4027980004,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0,0.000000,0.000000


/Users/vinniepalazeti/Desktop/github/GPS/examples
